## Google private Trends API

In this notebook I explored Google's private trends API, for which I was given special access. The python client for the API is not currently being supported by Google, however, so I just reverse engineered the format for the actual request payloads. This process produced two responses which are saved as Google-1.json and Google-2.json

In [1]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import googleapiclient.discovery
import configparser
cfg = configparser.ConfigParser()
cfg.read("secrets.ini")
my_key=cfg.get("default", "gtrends_apikey")

In [6]:
# didnt end up using this

SERVER = 'https://trends.googleapis.com'
VERSION = 'v1beta'
DISCOVERY_URL_SUFFIX = '/$discovery/rest?version=' + VERSION
DISCOVERY_URL = SERVER + DISCOVERY_URL_SUFFIX

In [12]:
# nor this

service = googleapiclient.discovery.build('trends',VERSION,developerKey=my_key
                                         , discoveryServiceUrl=DISCOVERY_URL)


Here is the format for the final payloads, with _MY_KEY_ replaced with the key, of course. 

    https://www.googleapis.com/trends/v1beta/timelinesForHealth?terms=flu&terms=cough&terms=cold&time.startDate=2014-01-01&time.endDate=2024-01-01&timelineResolution=week&geo=US-MI-563&key=_MY_KEY_&alt=json

A potential future viewer of this notebook might note this: Google limits its response to 2,000 items. You must choose your timeframes carefully, because if your number of weeks/months multiplied by the number of terms ends up being more than 2,000 you will get an error. For my project, I wanted data between 2004-2024. I made these two requests in increments of 10 years - hence the two output json responses. 

In [20]:
import json

In [93]:
with open('Google-1.json') as file:
    file_contents_1 = json.load(file)

In [96]:
pd.json_normalize(file_contents_1['flu'], meta=['value','date'])

,value,date
0,2000.695612,Jan 05 2014
1,1646.594980,Jan 12 2014
2,1323.911950,Jan 19 2014
3,1115.730179,Jan 26 2014
4,941.908677,Feb 02 2014
...,...,...
507,1277.860262,Sep 24 2023
508,1339.602284,Oct 01 2023
509,1242.949688,Oct 08 2023
510,1254.850155,Oct 15 2023


In [97]:
dd = pd.json_normalize(file_contents_1['flu'], meta=['value','date'])

In [98]:
dd['date'] = pd.to_datetime(dd['date'])

In [99]:
dd.set_index('date').rename(columns={'value':'flu'})

,flu
date,
2014-01-05,2000.695612
2014-01-12,1646.594980
2014-01-19,1323.911950
2014-01-26,1115.730179
2014-02-02,941.908677
...,...
2023-09-24,1277.860262
2023-10-01,1339.602284
2023-10-08,1242.949688


In [100]:
dd['cough'] = pd.json_normalize(file_contents_1['cough'], meta=['value','date'])['value']

In [101]:
dd['cold'] = pd.json_normalize(file_contents_1['cold'], meta=['value','date'])['value']

In [102]:
dd.rename(columns={'value':'flu'},inplace=True)

In [103]:
dd = dd.set_index('date')

In [92]:
ff = dd.copy()

In [ ]:
with open('Google-2.json') as file:
    file_contents_1 = json.load(file)

dd = pd.json_normalize(file_contents_1['flu'], meta=['value','date'])

dd['date'] = pd.to_datetime(dd['date'])
dd.set_index('date').rename(columns={'value':'flu'})
dd['cough'] = pd.json_normalize(file_contents_1['cough'], meta=['value','date'])['value']
dd['cold'] = pd.json_normalize(file_contents_1['cold'], meta=['value','date'])['value']
dd.rename(columns={'value':'flu'},inplace=True)
dd = dd.set_index('date')

In [104]:
dd

,flu,cough,cold
date,,,
2014-01-05,2000.695612,790.556033,4884.827197
2014-01-12,1646.594980,711.802781,3042.352004
2014-01-19,1323.911950,721.528106,3523.406869
2014-01-26,1115.730179,705.142613,3479.188166
2014-02-02,941.908677,663.683184,3066.753330
...,...,...,...
2023-09-24,1277.860262,1191.822409,3689.433580
2023-10-01,1339.602284,1228.294742,3710.771255
2023-10-08,1242.949688,1248.016096,3724.331102


In [105]:
ff

,flu,cough,cold
date,,,
2004-01-04,880.264644,372.022059,3881.784871
2004-01-11,709.409348,331.229860,3790.629287
2004-01-18,685.939707,289.228760,3744.224168
2004-01-25,1217.871215,302.004747,3659.188946
2004-02-01,1140.883644,299.740848,3096.137232
...,...,...,...
2013-12-01,772.012600,677.383113,3306.040851
2013-12-08,815.813262,708.972957,3592.284939
2013-12-15,957.099765,719.281124,3132.904581


In [110]:
google_df = ff.reset_index().merge(dd.reset_index(), how='outer')

In [111]:
google_df.to_csv('weekly google .csv')